In [ ]:
"""Notebok loads & explores the dataset.
"""

In [ ]:
from config import config
from data_processing.utils import viewer
import pathlib
import trimesh
import random
import numpy as np
import math

In [ ]:
SEED = 42
np.random.seed(SEED)

# Load the dataset

In [ ]:
# Load global config.
config_file = config.Config()

In [ ]:
dataset_train_path = pathlib.Path(config_file.config['dataset']['train'])
dataset_test_path = pathlib.Path(config_file.config['dataset']['test'])

In [ ]:
# Create a dict with class names & their indices.
folders = [dir.stem for dir in sorted(dataset_train_path.iterdir()) if dir.is_dir()]
classes = {folder: i for i, folder in enumerate(folders)};

In [ ]:
classes

# Visualize an example file

In [ ]:
example_path = dataset_train_path / 'motor/00001173.obj'

In [ ]:
example = trimesh.load(example_path, force='mesh')

In [ ]:
viz = viewer.Viewer()
viz.add_mesh(example)
viz.add_pc(example.vertices, size=2.5)
viz.show()

# Add extra points to the point clouds

In [ ]:
# This step is done as the mesh vertices alone do not give enough spatial information.

In [ ]:
pc, _ = trimesh.sample.sample_surface(example, 1000, seed=SEED)

In [ ]:
viz = viewer.Viewer()
viz.add_pc(pc, size=2.5)
viz.show()

# Normalize & augment the data.

In [ ]:
def normalize_pc(pc:np.ndarray) -> np.ndarray:
    # TODO(vice) Check the paper for name of the normalization
    """Normalizes point cloud.

    Args:
        pc: Input point cloud.

    Returns:
        Normalized point cloud.
    """
    if len(pc.shape) != 2:
        print('Invalid point cloud!')
        return np.array([])
    pc_norm = pc - np.mean(pc, axis=0)
    pc_norm /= np.max(np.linalg.norm(pc_norm, axis=1))
    
    return pc_norm

In [ ]:
pc_norm = normalize_pc(pc)

In [ ]:
viz = viewer.Viewer()
viz.add_pc(pc_norm, size=2.5)
viz.show()

In [ ]:
def apply_random_rotation_z(pc:np.ndarray) -> np.ndarray:
    """Applies random rotation around the z axis
    to the input point cloud.

    Args:
        pc: Input point cloud.

    Returns:
        Rotated point cloud.
    """
    theta = np.random.random(1) * 2 * math.pi
    rot_matrix = np.array([[math.cos(theta), -math.sin(theta), 0],
                           [math.sin(theta), math.cos(theta), 0],
                           [0, 0, 1]])
      
    pc_rot = rot_matrix.dot(pc.T).T

    return pc_rot

In [ ]:
pc_rot = apply_random_rotation_z(pc_norm)

In [ ]:
viz = viewer.Viewer()
viz.add_pc(pc_rot, size=2.5)
viz.show()

In [ ]:
def add_jitter(pc:np.ndarray) -> np.ndarray:
    """Applied random jitter to the point cloud.

    Args:
        pc: Input point cloud.

    Returns:
        Point cloud with added noise.
    """
    jitter = np.random.normal(0, 0.02, (pc.shape))
    pc_noisy = pc + jitter
    
    return pc_noisy

In [ ]:
pc_noisy = add_jitter(pc_rot)

In [ ]:
viz = viewer.Viewer()
viz.add_pc(pc_noisy, size=2.5)
viz.show()